In [1]:
import os
import re
import sys
import pandas as pd
sys.path.insert(0, './_modules')
from db_functions import TheDB
#from logging_module import getlogger
from logging_module_v2 import StartLogger
from regex_module import FindEmail, FindNumbersBraket, FindLawsuit, FindCaseId

In [2]:
## bots BD connections
conn_dict = {'host':'localhost', 'database': 'brspcqadev',
             'user':'root', 'password':'quito2020'}

cqdb = TheDB(conn_dict)

In [3]:
data_path = os.path.join(os.getcwd(), '_data_output')

In [4]:
os.listdir(data_path)

['tmail_data_clean_bigrams_cluster_3_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_22_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_12_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-04-01.csv',
 'mail_data_clean_bigrams.csv',
 'tmail_data_clean_bigrams_cluster_16_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_4_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-10.csv',
 'tmail_data_clean_bigrams_cluster_18_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_15_2020-04-05.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-04.csv',
 'tmail_data_clean_bigrams_cluster_20_2020-03-01.csv',
 'tmail_data_clean_bigrams_cluster_9_2020-03-26.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-04.csv',
 'MainTrix_BoletosAds.csv',
 'mail_data_clean_bigrams_optimalcluster.csv',
 'tmail_data_clean_bigrams_cluster_2_2020-03-01.csv',
 'mail_data_clean_bigrams_optimalcluster_2020-04-01.csv',
 'tmail_data_clean_bigrams_cluster_8_2020-03-01.csv',
 'oficios_google_cl

In [22]:
df = pd.read_csv(os.path.join(data_path, 'main_trix_clean.csv'), sep = '\t')
print(df.columns)
df = df.rename(columns = {'Case # (DLBCA and Google reference)':'case_id',\
                     'line':'line_id', 'Lawsuit Number':'external_ref_number', 'Deadline':'deadline'})
df = df.loc[:, ['line_id', 'case_id', 'external_ref_number', 'deadline']]
df['external_ref_number'] = df['external_ref_number'].apply(FindLawsuit)
df['case_id'] = df['case_id'].apply(FindCaseId)

Index(['line', 'Case # (DLBCA and Google reference)', 'UID',
       'Date of the Court Orders', 'Court', 'same court', 'Lawsuit Number',
       'CP?', 'Summary of the request/what to produce [internal team rec]',
       'STQ confirm scope of production', 'Deadline',
       'DLBCA's suggested answer', 'Previous Orders and Answers',
       'Google's proposed answer', 'LIS: \nInformatin about case',
       'LIS: \nPolicy and content check',
       'LIS: \nGoogle Inc answer sent to authority? which one?',
       'LIS:\nPriority', 'LIS:\nAssignee', 'Status',
       'Status - writ of mandamus', 'Law Firm'],
      dtype='object')


In [17]:
df

,line_id,case_id,external_ref_number,deadline
0,1,,0009460-19.2013.403.6181,2014-02-28 00:00:00
1,2,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00
2,3,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00
3,4,5-6191000002205,,2013-11-22 00:00:00
4,5,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00
...,...,...,...,...
7614,7740,7-9812000028504,5075998-22.2019.4.04.7000,2019-12-24 00:00:00
7615,7741,6-8938000029130,0330591-87.2019.8.05.0001,2019-12-30 00:00:00
7616,7742,0-5628000028599,505-23.2019.4.01.3601,2020-01-03 00:00:00
7617,7743,0-5628000028599,505-23.2019.4.01.3601,2020-12-24 00:00:00


In [25]:
## input some date, since those fields are manadatory
df['date_received'] = '2020-08-12'
df['date_received_cq'] = '2020-08-12'

In [40]:
df['line_id'].str.isdigit()

0       True
1       True
2       True
3       True
4       True
        ... 
7614    True
7615    True
7616    True
7617    True
7618    True
Name: line_id, Length: 7619, dtype: object

In [45]:
df = df[df['line_id'].str.isdigit() == True]
df['line_id'] = df['line_id'].astype(int)

In [60]:
df_db_trix = cqdb.GetTable('Trix')
df_db_trix

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,35,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,36,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,37,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,38,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,39,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
5,40,,0137528-90.2013.8.20.0001,2014-01-07 00:00:00,2020-08-12,2020-08-12
6,41,7-2587000000237,31820-28.2012.4.01.3500,2013-11-26 00:00:00,2020-08-12,2020-08-12
7,42,1-9094000002174,0001129-37.2012.8.20.0115,2013-11-27 00:00:00,2020-08-12,2020-08-12
8,43,6-3783000002820,12509-97.2013.811.0042,2014-03-05 00:00:00,2020-08-12,2020-08-12
9,44,3-4834000002843,5028182-54.2013.404.7000,2014-02-13 00:00:00,2020-08-12,2020-08-12


In [46]:
df['line_id'] = df['line_id'].astype(int) + df_db_trix['line_id'].max()

In [54]:
df = df.drop_duplicates(subset = ['line_id'])

In [49]:
cqdb.TablesDataType('Trix')

,0,1,2,3,4,5
0,line_id,int(11),NO,PRI,None,
1,case_id,longtext,YES,,None,
2,date_received,datetime(6),NO,,None,
3,date_received_cq,datetime(6),NO,,None,
4,auth_last_name,longtext,YES,,None,
5,external_ref_number,longtext,YES,,None,
6,info_case,longtext,YES,,None,
7,case_country,longtext,YES,,None,
8,email_response,longtext,YES,,None,
9,deadline,datetime,YES,,None,


In [64]:
df_db_trix = cqdb.GetTable('Trix')

In [65]:
df_db_trix

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq,auth_last_name
0,35,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12,None
1,36,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12,None
2,37,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12,None
3,38,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12,None
4,39,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12,None
5,40,,0137528-90.2013.8.20.0001,2014-01-07 00:00:00,2020-08-12,2020-08-12,None
6,41,7-2587000000237,31820-28.2012.4.01.3500,2013-11-26 00:00:00,2020-08-12,2020-08-12,None
7,42,1-9094000002174,0001129-37.2012.8.20.0115,2013-11-27 00:00:00,2020-08-12,2020-08-12,None
8,43,6-3783000002820,12509-97.2013.811.0042,2014-03-05 00:00:00,2020-08-12,2020-08-12,None
9,44,3-4834000002843,5028182-54.2013.404.7000,2014-02-13 00:00:00,2020-08-12,2020-08-12,None


In [63]:
df_db_trix

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,35,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,36,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,37,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,38,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,39,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
5,40,,0137528-90.2013.8.20.0001,2014-01-07 00:00:00,2020-08-12,2020-08-12
6,41,7-2587000000237,31820-28.2012.4.01.3500,2013-11-26 00:00:00,2020-08-12,2020-08-12
7,42,1-9094000002174,0001129-37.2012.8.20.0115,2013-11-27 00:00:00,2020-08-12,2020-08-12
8,43,6-3783000002820,12509-97.2013.811.0042,2014-03-05 00:00:00,2020-08-12,2020-08-12
9,44,3-4834000002843,5028182-54.2013.404.7000,2014-02-13 00:00:00,2020-08-12,2020-08-12


In [58]:
df[:10]

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,35,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,36,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,37,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,38,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,39,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
5,40,,0137528-90.2013.8.20.0001,2014-01-07 00:00:00,2020-08-12,2020-08-12
6,41,7-2587000000237,31820-28.2012.4.01.3500,2013-11-26 00:00:00,2020-08-12,2020-08-12
7,42,1-9094000002174,0001129-37.2012.8.20.0115,2013-11-27 00:00:00,2020-08-12,2020-08-12
8,43,6-3783000002820,12509-97.2013.811.0042,2014-03-05 00:00:00,2020-08-12,2020-08-12
9,44,3-4834000002843,5028182-54.2013.404.7000,2014-02-13 00:00:00,2020-08-12,2020-08-12


In [59]:
df_db_trix2 = cqdb.InsertDB(df[:10],\
                            'Trix', key_column = '' , return_table = True, append_table = False)

Original length: 10
Final length: 44
Success! Data writtten in db!


In [62]:
df_db_trix2[0]

,line_id,case_id,external_ref_number,deadline,date_received,date_received_cq
0,35,,0009460-19.2013.403.6181,2014-02-28 00:00:00,2020-08-12,2020-08-12
1,36,6-9080000028545,0002618-91.2011.403.6181,2013-11-28 00:00:00,2020-08-12,2020-08-12
2,37,9-5821000001559,0007067-24.2013.403.6181,2013-10-22 00:00:00,2020-08-12,2020-08-12
3,38,5-6191000002205,,2013-11-22 00:00:00,2020-08-12,2020-08-12
4,39,9-9709000000828,0076690-72.2011.805.0001,2013-10-21 00:00:00,2020-08-12,2020-08-12
5,40,,0137528-90.2013.8.20.0001,2014-01-07 00:00:00,2020-08-12,2020-08-12
6,41,7-2587000000237,31820-28.2012.4.01.3500,2013-11-26 00:00:00,2020-08-12,2020-08-12
7,42,1-9094000002174,0001129-37.2012.8.20.0115,2013-11-27 00:00:00,2020-08-12,2020-08-12
8,43,6-3783000002820,12509-97.2013.811.0042,2014-03-05 00:00:00,2020-08-12,2020-08-12
9,44,3-4834000002843,5028182-54.2013.404.7000,2014-02-13 00:00:00,2020-08-12,2020-08-12


In [69]:
cqdb.TablesDataType('Trix')

,0,1,2,3,4,5
0,line_id,int(11),NO,PRI,None,
1,case_id,text,YES,,None,
2,external_ref_number,text,YES,,None,
3,deadline,text,YES,,None,
4,date_received,text,YES,,None,
5,date_received_cq,text,YES,,None,
6,auth_last_name,longtext,YES,,None,
7,info_case,longtext,YES,,None,
8,case_country,longtext,YES,,None,
9,email_response,longtext,YES,,None,
